## **Instagram Scraping with Selenium and Instaloader (filtered for 2025 posts)** 📸
_by hazellenuts_

### 🗃️ Import Libraries

In [ ]:
import pandas as pd
import time
import random
from datetime import datetime
import warnings
import instaloader

warnings.filterwarnings("ignore")

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC



### Chromedriver Configuration

In [2]:
chromedriver_path = r"C:\\Users\\chromedriver.exe"
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

### Log in to your Instagram Account

In [ ]:
user_ig = "[your_ig_username]" #replace with your ig username
pass_ig = "[your_ig_password]" #replace with your ig password
url_target = "https://www.instagram.com/[ig_username]}" #replace with targeted url
driver.get("https://www.instagram.com/")
time.sleep(random.randint(2, 4))

username = driver.find_element(By.CSS_SELECTOR, "input[name='username']")
password = driver.find_element(By.CSS_SELECTOR, "input[name='password']")
username.clear()
password.clear()
username.send_keys(user_ig)
password.send_keys(pass_ig)

login_btn = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
login_btn.click()
time.sleep(5)

driver.get(url_target)
time.sleep(5)


### Scrape url

In [ ]:
url_postingan_set = set()
scroll_times = 0
MAX_SCROLL = 1 #replace with your desired max scroll times

while scroll_times < MAX_SCROLL:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.randint(2, 4))

    posts = driver.find_elements(By.CSS_SELECTOR, "a[href*='/p/']")
    for post in posts:
        url_postingan_set.add(post.get_attribute("href"))

    scroll_times += 1

url_postingan_list = list(url_postingan_set)
print(f"Total post links found: {len(url_postingan_list)}")


Total post links found: 24


### Fetch posting dates (only 2025)

In [5]:
urls_2025 = []

for url in url_postingan_list:
    driver.get(url)
    time.sleep(random.randint(2, 4))
    try:
        time_elem = driver.find_element(By.TAG_NAME, "time")
        post_date = time_elem.get_attribute("datetime")

        post_datetime = datetime.fromisoformat(post_date.replace("Z", ""))
        post_year = post_datetime.year

        if post_year == 2025:
            formatted_date = post_datetime.strftime("%d %B %Y, %H:%M")
            urls_2025.append([url, formatted_date])

    except Exception as e:
        print(f"Error found {url}: {e}")
        continue

In [6]:
df_2025 = pd.DataFrame(urls_2025, columns=['url', 'post_date'])
df_2025 = df_2025.drop_duplicates().reset_index(drop=True)
df_2025.head()

,url,post_date
0,https://www.instagram.com/unjcareercenter/p/DM...,"22 July 2025, 01:54"
1,https://www.instagram.com/unjcareercenter/p/DL...,"11 July 2025, 02:42"
2,https://www.instagram.com/unjcareercenter/p/DM...,"22 July 2025, 08:05"
3,https://www.instagram.com/unjcareercenter/p/DN...,"07 August 2025, 06:53"
4,https://www.instagram.com/unjcareercenter/p/DN...,"19 August 2025, 02:55"


In [7]:
# close driver
driver.quit()


### Fetch captions, like count, comment count using Instaloader

In [8]:
L = instaloader.Instaloader()
L.login(user_ig, pass_ig)

In [ ]:
captions = []
likes = []
comments_count = []

for url in df_2025["url"]:
    try:
        shortcode = url.split("/p/")[1].split("/")[0]
        post = instaloader.Post.from_shortcode(L.context, shortcode)

        captions.append(post.caption)
        likes.append(post.likes)
        comments_count.append(post.comments)

    except Exception as e:
        print(f"failed to fetch data from {url}: {e}")
        captions.append(None)
        likes.append(None)
        comments_count.append(None)

df_2025["caption"] = captions
df_2025["likes"] = likes
df_2025["jumlah_komen"] = comments_count

df_2025.head()

,url,post_date,caption,likes,jumlah_komen
0,https://www.instagram.com/unjcareercenter/p/DM...,"22 July 2025, 01:54",We Are Hiring!\n.\nBergabunglah Bersama Tim IS...,10,0
1,https://www.instagram.com/unjcareercenter/p/DL...,"11 July 2025, 02:42",📣 [LOWONGAN KERJA – KHUSUS ALUMNI UNJ]\n✨ Berg...,50,0
2,https://www.instagram.com/unjcareercenter/p/DM...,"22 July 2025, 08:05","Pernah ngerasa kelas udah aktif, tapi siswa be...",13,0
3,https://www.instagram.com/unjcareercenter/p/DN...,"07 August 2025, 06:53",Lulus Kuliah tapi bingung mau ngapain?\nAtau u...,70,1
4,https://www.instagram.com/unjcareercenter/p/DN...,"19 August 2025, 02:55",📢Untuk warga DKI Jakarta dan sekitarnya yang s...,84,0


### Save to Excel

In [10]:
output_file = "scraped.xlsx"
df_2025.to_excel(output_file, index=False)

print(f"file saved as {output_file}")

file saved as scraped.xlsx
